## Seldon V2 Non Kubernetes Local Experiment Examples


### Model Experiment

We will use two SKlearn Iris classification models to illustrate experiments.

In [50]:
!cat ./models/sklearn1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


In [51]:
!cat ./models/sklearn2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris2
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  requirements:
  - sklearn


Load both models.

In [226]:
!seldon model load -f ./models/sklearn1.yaml
!seldon model load -f ./models/sklearn2.yaml

{}
{}


Wait for both models to be ready.

In [227]:
!seldon model status iris -w ModelAvailable
!seldon model status iris2 -w ModelAvailable

{}
{}


In [228]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [229]:
!seldon model infer iris2 -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::50]


Create an experiment that modifies the iris model to add a second model splitting traffic 50/50 between the two.

In [230]:
!cat ./experiments/ab-default-model.yaml 

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: experiment-sample
spec:
  default: iris
  candidates:
  - modelName: iris
    weight: 50
  - modelName: iris2
    weight: 50


Start the experiment.

In [231]:
!seldon experiment start -f ./experiments/ab-default-model.yaml 

{}


Wait for the experiment to be ready.

In [232]:
!seldon experiment status experiment-sample -w | jq -M .

{
  "experimentName": "experiment-sample",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


Run a set of calls and record which route the traffic took. There should be roughly a 50/50 split.

In [233]:
!seldon model infer iris -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris2_1::20 :iris_1::30]


Show sticky session header `x-seldon-route` that is returned

In [237]:
!seldon model infer iris --show-headers \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

Request header Content-Type:[application/json]
Request header Seldon-Model:[iris]
Response header Ce-Requestid:[c5dca540-ae36-4e25-bae2-1d7baa0ff5c8]
Response header Date:[Sat, 20 Aug 2022 17:38:56 GMT]
Response header X-Envoy-Upstream-Service-Time:[2]
Response header X-Seldon-Route:[:iris_1:]
Response header Ce-Id:[c5dca540-ae36-4e25-bae2-1d7baa0ff5c8]
Response header Ce-Specversion:[0.3]
Response header Server:[envoy]
Response header Traceparent:[00-16b02e51bb04c2845156ff4968cc0623-7a9f354372a87ecc-01]
Response header Content-Length:[228]
Response header Content-Type:[application/json]
Response header Ce-Endpoint:[iris_1]
Response header Ce-Inferenceservicename:[mlserver]
Response header Ce-Source:[io.seldon.serving.deployment.mlserver]
Response header Ce-Type:[io.seldon.serving.inference.response]
Response header Ce-Modelid:[iris_1]
Response header X-Request-Id:[86e1ddab-b8dc-4204-9b53-1b09d521a377]
{
	"model_name": "iris_1",
	"model_version": "1",
	"id": "c5dca540-ae36-4e25-bae2-1d

Use sticky session key passed by last infer request to ensure same route is taken each time.

In [238]:
!seldon model infer iris -s -i 50 \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

map[:iris_1::50]


In [239]:
!seldon model infer iris --inference-mode grpc -s -i 50\
   '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' 

map[:iris_1::50]


Stop the experiment

In [240]:
!seldon experiment stop experiment-sample

{}


Unload both models.

In [241]:
!seldon model unload iris
!seldon model unload iris2

{}
{}


### Pipeline Experiment

In [242]:
!cat ./models/add10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: add10
spec:
  storageUri: "gs://seldon-models/triton/add10"
  requirements:
  - triton
  - python


In [243]:
!cat ./models/mul10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: mul10
spec:
  storageUri: "gs://seldon-models/triton/mul10"
  requirements:
  - triton
  - python


In [244]:
!seldon model load -f ./models/add10.yaml
!seldon model load -f ./models/mul10.yaml

{}
{}


In [245]:
!seldon model status add10 -w ModelAvailable
!seldon model status mul10 -w ModelAvailable

{}
{}


In [246]:
!cat ./pipelines/mul10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: pipeline-mul10
spec:
  steps:
    - name: mul10
  output:
    steps:
    - mul10


In [247]:
!cat ./pipelines/add10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Pipeline
metadata:
  name: pipeline-add10
spec:
  steps:
    - name: add10
  output:
    steps:
    - add10


In [248]:
!seldon pipeline load -f ./pipelines/add10.yaml
!seldon pipeline load -f ./pipelines/mul10.yaml

{}
{}


In [249]:
!seldon pipeline status pipeline-add10 -w PipelineReady 
!seldon pipeline status pipeline-mul10 -w PipelineReady 

{"pipelineName":"pipeline-add10","versions":[{"pipeline":{"name":"pipeline-add10","uid":"cc0hcueckehqk02vdf00","version":1,"steps":[{"name":"add10"}],"output":{"steps":["add10.outputs"]},"kubernetesMeta":{}},"state":{"pipelineVersion":1,"status":"PipelineReady","reason":"Created pipeline","lastChangeTimestamp":"2022-08-20T17:38:20.996143387Z"}}]}
{"pipelineName":"pipeline-mul10","versions":[{"pipeline":{"name":"pipeline-mul10","uid":"cc0hcueckehqk02vdf0g","version":1,"steps":[{"name":"mul10"}],"output":{"steps":["mul10.outputs"]},"kubernetesMeta":{}},"state":{"pipelineVersion":1,"status":"PipelineReady","reason":"Created pipeline","lastChangeTimestamp":"2022-08-20T17:38:21.629812312Z"}}]}


In [250]:
!seldon pipeline infer pipeline-add10 --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

{"outputs":[{"name":"OUTPUT","datatype":"FP32","shape":["4"],"contents":{"fp32Contents":[11,12,13,14]}}],"rawOutputContents":["AAAwQQAAQEEAAFBBAABgQQ=="]}


In [251]:
!seldon pipeline infer pipeline-mul10 --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

{"outputs":[{"name":"OUTPUT","datatype":"FP32","shape":["4"],"contents":{"fp32Contents":[10,20,30,40]}}],"rawOutputContents":["AAAgQQAAoEEAAPBBAAAgQg=="]}


In [252]:
!cat ./experiments/addmul10.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: addmul10
spec:
  default: pipeline-add10
  resourceType: pipeline
  candidates:
  - modelName: pipeline-add10
    weight: 50
  - modelName: pipeline-mul10
    weight: 50


In [253]:
!seldon experiment start -f ./experiments/addmul10.yaml 

{}


In [254]:
!seldon experiment status addmul10 -w | jq -M .

{
  "experimentName": "addmul10",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


In [255]:
!seldon pipeline infer pipeline-add10 -i 50 --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

map[:add10_1::22 :mul10_1::28 :pipeline-add10.pipeline::22 :pipeline-mul10.pipeline::28]


Use sticky session key passed by last infer request to ensure same route is taken each time.

In [256]:
!seldon pipeline infer pipeline-add10 --show-headers --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

Request metadata seldon-model:[pipeline-add10.pipeline]
{"outputs":[{"name":"OUTPUT","datatype":"FP32","shape":["4"],"contents":{"fp32Contents":[10,20,30,40]}}],"rawOutputContents":["AAAgQQAAoEEAAPBBAAAgQg=="]}
Response header x-forwarded-proto:[http]
Response header x-request-id:[5f845705-26ca-4702-a384-bb0596859b42]
Response header date:[Sat, 20 Aug 2022 17:39:36 GMT]
Response header server:[envoy]
Response header content-type:[application/grpc]
Response header x-envoy-upstream-service-time:[12]
Response header x-seldon-route:[:mul10_1: :pipeline-mul10.pipeline:]
Response header x-envoy-expected-rq-timeout-ms:[60000]


In [257]:
!seldon pipeline infer pipeline-add10 -s --show-headers --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

Request metadata x-seldon-route:[:mul10_1: :pipeline-mul10.pipeline:]
Request metadata seldon-model:[pipeline-add10.pipeline]
{"outputs":[{"name":"OUTPUT","datatype":"FP32","shape":["4"],"contents":{"fp32Contents":[10,20,30,40]}}],"rawOutputContents":["AAAgQQAAoEEAAPBBAAAgQg=="]}
Response header x-envoy-expected-rq-timeout-ms:[60000]
Response header x-seldon-route:[:mul10_1: :pipeline-mul10.pipeline:]
Response header x-envoy-upstream-service-time:[11]
Response header date:[Sat, 20 Aug 2022 17:39:39 GMT]
Response header server:[envoy]
Response header content-type:[application/grpc]
Response header x-request-id:[b8433074-40ac-43f1-b785-38bbff9855f8]
Response header x-forwarded-proto:[http]


In [258]:
!seldon pipeline infer pipeline-add10 -s -i 50 --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

map[:mul10_1::50 :pipeline-mul10.pipeline::50]


In [259]:
!cat ./models/add20.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: add20
spec:
  storageUri: "gs://seldon-models/triton/add20"
  requirements:
  - triton
  - python


In [260]:
!seldon model load -f ./models/add20.yaml

{}


In [261]:
!seldon model status add20 -w ModelAvailable

{}


In [262]:
!cat ./experiments/add1020.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Experiment
metadata:
  name: add1020
spec:
  default: add10
  candidates:
  - modelName: add10
    weight: 50
  - modelName: add20
    weight: 50


In [263]:
!seldon experiment start -f ./experiments/add1020.yaml

{}


In [264]:
!seldon experiment status add1020 -w | jq -M .

{
  "experimentName": "add1020",
  "active": true,
  "candidatesReady": true,
  "mirrorReady": true,
  "statusDescription": "experiment active",
  "kubernetesMeta": {}
}


In [265]:
!seldon model infer add10 -i 50  --inference-mode grpc \
  '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

map[:add10_1::26 :add20_1::24]


In [266]:
!seldon pipeline infer pipeline-add10 -i 100 --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

map[:add10_1::16 :add20_1::29 :mul10_1::55 :pipeline-add10.pipeline::45 :pipeline-mul10.pipeline::55]


In [269]:
!seldon pipeline infer pipeline-add10 --show-headers --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

Request metadata seldon-model:[pipeline-add10.pipeline]
{"outputs":[{"name":"OUTPUT","datatype":"FP32","shape":["4"],"contents":{"fp32Contents":[10,20,30,40]}}],"rawOutputContents":["AAAgQQAAoEEAAPBBAAAgQg=="]}
Response header x-request-id:[0bb5f3b1-435c-4c38-a8d5-26ec90f4b708]
Response header date:[Sat, 20 Aug 2022 17:40:20 GMT]
Response header server:[envoy]
Response header content-type:[application/grpc]
Response header x-envoy-upstream-service-time:[9]
Response header x-seldon-route:[:mul10_1: :pipeline-mul10.pipeline:]
Response header x-envoy-expected-rq-timeout-ms:[60000]
Response header x-forwarded-proto:[http]


In [270]:
!seldon pipeline infer pipeline-add10 -s --show-headers --inference-mode grpc \
 '{"model_name":"add10","inputs":[{"name":"INPUT","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[4]}]}' 

Request metadata seldon-model:[pipeline-add10.pipeline]
Request metadata x-seldon-route:[:mul10_1: :pipeline-mul10.pipeline:]
{"outputs":[{"name":"OUTPUT","datatype":"FP32","shape":["4"],"contents":{"fp32Contents":[10,20,30,40]}}],"rawOutputContents":["AAAgQQAAoEEAAPBBAAAgQg=="]}
Response header x-envoy-upstream-service-time:[9]
Response header x-envoy-expected-rq-timeout-ms:[60000]
Response header x-seldon-route:[:mul10_1: :pipeline-mul10.pipeline:]
Response header x-forwarded-proto:[http]
Response header x-request-id:[11a1ba4a-5fd9-4a5c-a924-77262aee4eb2]
Response header date:[Sat, 20 Aug 2022 17:40:23 GMT]
Response header server:[envoy]
Response header content-type:[application/grpc]


In [271]:
!seldon experiment stop addmul10
!seldon experiment stop add1020
!seldon pipeline unload pipeline-add10
!seldon pipeline unload pipeline-mul10
!seldon model unload add10
!seldon model unload add20
!seldon model unload mul10

{}
{}
{}
{}
{}
{}
{}
